In [2]:
# This cell is from the template notebook. It contains ipython magic and 
# various useful imports which are commonly used in this app.

%load_ext autoreload
%autoreload 2

import pandas as pd
import datetime
import numpy as np
import os
from acoustic_data_science import config, helpers
import matplotlib.pyplot as plt
import logging

In [5]:
def get_date_from_datestr(str): 
    return str[:10]

In [20]:
#dataframe with daily temperature data from Oceans2.0 website

data_file_name = "ShoreStation_MeteorologicalStation-Lufft_MeteorologicalStation_AirTemperature_20180801T113900Z_20190729T210100Z-NaN_clean.csv"
temp_data = pd.read_csv(os.path.join(config.external_data_path, data_file_name), skiprows=52)

#formatting 
temp_data.drop(temp_data.index[[0]], inplace=True)                                 #drop first row with no data 
temp_data = temp_data.iloc[: , :-3]                                                #remove last 3 columns
temp_data.set_axis(['Time', 'Air Temp', 'Flag'], axis=1, inplace=True)             #rename columns
temp_data['Date'] = temp_data['Time'].apply(get_date_from_datestr)                                  #just get date

temp_data['Date'] = pd.to_datetime(temp_data['Date'], dayfirst=True)               #convert to timestamp  
temp_data['Air Temp'] = temp_data['Air Temp'].astype(float)                        #convert to float 
daily_temp = temp_data.groupby(by='Date').mean()                                   #new df for daily average temp 
daily_temp.reset_index(inplace=True)                                               #reset index

#add temperature change column 
daily_temp['Temp Change'] = np.nan
for i in range(0, len(daily_temp)-1): 
    daily_temp.loc[i+1, 'Temp Change'] = (daily_temp.loc[i+1, 'Air Temp']) - (daily_temp.loc[i, 'Air Temp'])
    
daily_temp = daily_temp.drop(columns=['Flag', 'Temp Change'])
daily_temp = daily_temp.rename(columns={"Date": "timestamp", "Air Temp": "mean_temperature"})
daily_temp

,timestamp,mean_temperature
0,2018-08-01,6.600901
1,2018-08-02,6.107467
2,2018-08-03,6.299008
3,2018-08-04,8.332302
4,2018-08-05,7.277796
...,...,...
358,2019-07-25,5.329940
359,2019-07-26,5.365808
360,2019-07-27,6.282461
361,2019-07-28,7.126437


In [21]:
daily_temp.to_feather(config.processed_data_path + '/daily_temperature_cambay_shorestation.feather')
